In [ ]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import MinMaxScaler
import ast
import nltk
nltk.download('punkt')
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

In [ ]:
'''If executed in Google Colab, uncomment the following lines'''
#from google.colab import drive
#drive.mount('/content/drive')

#import os
#os.chdir('/content/drive/MyDrive/LLM_CreditorRRPrediction')

In [ ]:
#PART = "presentation_summary"
#PART = "qna_summary"
PART = "analysts_summary"

# Initialize columns for each emotion
ORIGINAL = True

df = pd.read_csv("transcripts/credit_df.csv", delimiter="|")

if ORIGINAL:
    emotions = [
        'admiration', 
        #'amusement', 
        'anger', 
        'annoyance', 
        'approval', 
        'caring', 
        'confusion', 
        'curiosity', 
        'desire',
        'disappointment', 
        'disapproval', 
        #'disgust', 
        'embarrassment', 
        'excitement', 
        'fear', 
        #'gratitude', 
        #'grief',
        'joy', 
        #'love', 
        'nervousness', 
        'optimism', 
        'pride', 
        'realization', 
        'relief', 
        'remorse', 
        'sadness', 
        'surprise'
    ]
else:
    emotions = ['positive', 'negative', 'ambiguous']

In [ ]:
'''Follow intructions on https://github.com/monologg/GoEmotions-pytorch to download the model and tokenizer'''

'''
from transformers import BertTokenizer
from model import BertForMultiLabelClassification
from multilabel_pipeline import MultiLabelPipeline

original = True

if original:
  tokenizer = BertTokenizer.from_pretrained("monologg/bert-base-cased-goemotions-original")
  model = BertForMultiLabelClassification.from_pretrained("monologg/bert-base-cased-goemotions-original")
else:
  tokenizer = BertTokenizer.from_pretrained("monologg/bert-base-cased-goemotions-group")
  model = BertForMultiLabelClassification.from_pretrained("monologg/bert-base-cased-goemotions-group")

goemotions = MultiLabelPipeline(
    model=model,
    tokenizer=tokenizer,
    threshold=0.3
)

df[f'{PART}_scores'] = None

# Process each earnings call
for idx, transcript in enumerate(df[PART]):
  try:
    call = nltk.sent_tokenize(transcript)  # Tokenize sentences from the cleaned text
    scores = goemotions(call)  # Assuming `goemotions` processes the sentences and returns emotion scores
    df.at[idx, f'{PART}_scores'] = scores
    print(idx)
  except:
    df.at[idx, f'{PART}_scores'] = []
    print(idx)

df.to_csv(f'goemotions/emotion_scores_{PART}.csv', sep='|')
'''

In [ ]:
scores = pd.read_csv(f'goemotions/emotion_scores_{PART}.csv', delimiter='|')

# Convert strings in 'presentation_summary_scores' to actual lists
scores[f'{PART}_scores'] = scores[f'{PART}_scores'].apply(ast.literal_eval)

scores.head()

In [ ]:
# Initialize columns for each emotion in the DataFrame
for emotion in emotions:
    scores[emotion] = 0.0

# Process each transcript's scores and calculate the emotion features
for idx, row in scores.iterrows():
    # Initialize a dictionary to hold total emotion scores
    emotion_scores = {emotion: 0.0 for emotion in emotions}
    
    # Get the list of sentence scores
    sentence_scores = row[f'{PART}_scores']
    sentence_count = len(sentence_scores)
    
    # Accumulate scores for each emotion
    for entry in sentence_scores:
        for label, score in zip(entry['labels'], entry['scores']):
            if label in emotion_scores:
                emotion_scores[label] += score

    # Normalize the scores by the number of sentences (average per sentence)
    for emotion, total_score in emotion_scores.items():
        scores.at[idx, emotion] = total_score / sentence_count if sentence_count > 0 else 0.0

In [ ]:
bond_level = pd.read_csv('transcripts/credit_df.csv', delimiter='|')  

# merge output_df with transcripts on 'transcript_number' and index
bond_level = pd.merge(bond_level,
                          scores[emotions + ['call_ID']],
                          on=['call_ID'],
                          how='left')

# normalize all emotion columns
scaler = MinMaxScaler()
bond_level[emotions] = scaler.fit_transform(bond_level[emotions])

checkpoint = bond_level.copy()

In [ ]:
#checkpoint.to_csv(f'goemotions/{PART}_final.csv', sep='|')
checkpoint = pd.read_csv(f'goemotions/{PART}_final.csv', delimiter='|')

In [ ]:
supporting_features_1 = [
    'CBOE DJIA Volatility Index',
    'NASDAQ 100 Index return',
    'Manufacturers inventories to sales ratio',
    '30 year conventional mortgage rate',
    'Communication Services', 
    'Consumer Discretionary', 
    'Senior secured',  
    'Time to maturity',  
    'Equity value',
    'CDS availability',
    'ActIndustryDistress1',
    'ActIndustryDistress2',
    'Offering amount',
    'Volume',
    'Industrials','Consumer Staples','Financials','Energy','Health Care','Utilities','Information Technology','Real Estate'
]

supporting_features_2 = [
    'Default barrier',
    'LTDIssuance2',
    'Intangibility',
    'Receivables1',
]

# Select the supporting features, nlp_lables, and RR from final_df
final_df = checkpoint[['RR']
                    #+ supporting_features_1
                    #+ supporting_features_2
                    + emotions
                    ]

# replace #DIV/0! & Nan with 0
# Replace '#DIV/0!' with NaN
final_df.replace('#DIV/0!', np.nan, inplace=True)
final_df = final_df.fillna(0)

# make sure all values are numeric except for the Date column
final_df = final_df.apply(pd.to_numeric, errors='ignore')

In [ ]:
# In-Sample-Regression
y_train, y_test = final_df['RR'], final_df['RR']
X_train, X_test = final_df.drop(columns=['RR']), final_df.drop(columns=['RR'])

In [ ]:
# Bond data 1 + LLM features

X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

# Fit the model
model = sm.OLS(y_train, X_train).fit()

# Print the summary of the model which includes p-values and significance levels
print(model.summary())

# Make predictions
y_pred = model.predict(X_test)

# Compute and print evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {np.sqrt(mse)}")
print(f"R-squared: {r2}")

# other metric
# Calculate the residuals
residuals = y_test - y_pred

# Plot the residuals
plt.figure(figsize=(10, 5))
plt.scatter(y_test, residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Recovery Rate')
plt.ylabel('Residuals')

# Display the plot
plt.show()

In [ ]:
# Compute variance inflation factor 
vif = pd.DataFrame()
vif["Features"] = X_train.columns
vif["VIF"] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
vif = vif.sort_values(by='VIF', ascending=False)

# Display the VIF
print(vif)

In [ ]:
# select significant features
model.significance = model.pvalues[model.pvalues < 0.05].index

# build a graph to show the importance of each feature
importances = model.params[1:]
importances = importances.sort_values()

# Sort the importance values based on absolute values, not just positive or negative
top_10_importances = importances.abs().sort_values(ascending=False).head(15)

# reduce importances to only the top 10
importances = importances[top_10_importances.index]
importances = importances.sort_values()

# color significant features 0/150/130 and others in grey
colors = ['#009682' if feature in model.significance else 'grey' for feature in importances.index]

plt.figure(figsize=(10, 5))
plt.barh(importances.index, importances.values, color=colors)
# Create a custom legend
handles = [plt.Rectangle((0, 0), 1, 1, color='#009682'), plt.Rectangle((0, 0), 1, 1, color='grey')]
labels = ['Significant', 'Not Significant']
plt.legend(handles, labels)
plt.xlabel('Impact')
plt.ylabel('Features')
plt.title('Feature Impact on Recovery Rate')
plt.show()

In [ ]:
import matplotlib.patches as mpatches

# Filter significant features
significant_features = model.pvalues[model.pvalues < 0.05].index
importances = model.params[significant_features]  # Only significant features
importances = importances[1:]

# Sort importances by their absolute value, and select the top 10
top_10_importances = importances.abs().sort_values(ascending=False).head(11)
importances = importances[top_10_importances.index]
importances = importances.sort_values()  # Sort the values for better visualization

# Define emotion types (assuming the feature names match these emotion labels)
emotion_types = {
    'relief': 'positive',
    'remorse': 'negative',
    'nervousness': 'negative',
    'desire': 'positive',
    'fear': 'strong_negative',
    'excitement': 'positive',
    'confusion': 'negative',
    'pride': 'positive',
    'annoyance': 'negative',
    'gratitude': 'positive',
    'anger': 'strong_negative',
    'optimism': 'positive',
    'sadness': 'negative', 
    'approval': 'positive',
    'caring': 'positive',
    'disappointment': 'negative',
    'curiosity': 'positive',
    'surprise': 'positive',
    'admiration': 'positive',
    'embarrassment': 'negative',
    'realization': 'positive',
    'disapproval': 'negative',
    'joy': 'positive'
}

# Map colors to each emotion type
color_mapping = {
    'positive': 'green',
    'strong_negative': 'red',
    'negative': 'gray',
}

# Apply the color mapping to the top 10 significant features
colors = [color_mapping[emotion_types[emotion]] for emotion in importances.index]

# Create a horizontal bar plot for the top 10 significant features
plt.figure(figsize=(10, 5))
plt.barh(importances.index, importances.values, color=colors)

# Add labels and title
plt.xlabel('Importance')
plt.ylabel('Features')
plt.title('Remaining Significant Features and their Impact')

# Add a legend to the top left corner
handles = [mpatches.Patch(color=color_mapping[emotion], label=emotion) for emotion in color_mapping]
plt.legend(handles=handles, loc='upper left')

# Show the plot
plt.show()